# Libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import requests_cache
import lxml.html as lx
from save_load import save_load
save_load = save_load()

requests_cache.install_cache("internal/mycache")

# Data Retrieval

Example url used for read_html_table:

https://www.arb.ca.gov/aqmis2/display.php?param=OZONE&units=007&year=2018&report=SITE1YR&statistic=DMAX&site=2460&ptype=aqd

Example url used for read_links:

https://www.arb.ca.gov/aqmis2/display.php?param=OZONE&units=007&year=2018&county_name=--COUNTY--&basin=--AIR+BASIN--&latitude=A-Whole+State&report=YRINV&order=s.name&submit=Retrieve+Data&ptype=aqd&std15=

read_links("OZONE", "007", 2005, "https://www.arb.ca.gov/aqmis2/display.php?param=OZONE&units=007&year=2005&county_name=--COUNTY--&basin=--AIR+BASIN--&latitude=A-Whole+State&report=YRINV&order=s.name&submit=Retrieve+Data&ptype=aqd&std15=")

Example url used in get_all_links:

https://www.arb.ca.gov/aqmis2/display.php?param=OZONE&units=007&year=2018&county_name=--COUNTY--&basin=--AIR+BASIN--&latitude=A-Whole+State&report=YRINV&order=s.name&submit=Retrieve+Data&ptype=aqd&std15=

In [2]:
def read_html_table(url):
    """
    Given URL to one AQMIS page for a site, find the table that contains daily ozone for a given year,
    (average ozone for each hour is collected, then the table is max ozone out of all hours in the day)
    return a pandas data frame
    """
    response = requests.get(url)
    response.raise_for_status()
    html = response.content
    df_list = pd.read_html(html)
    # table with index 2 is what we want
    return df_list[2]


def reformat_table(df):
    """
    Given a data frame taken from the page of one AQMIS site,
    remove unneeded rows and columns and return it
    """
    return df.drop([0, 1, 33, 34, 35]).drop([0], axis = 1)


def read_links(param, units, year, url):
    """
    Given a URL to the AQMIS site containing links to tables for all locations,
    find all the links to the proper tables and return them in a list
    """
    response = requests.get(url)
    response.raise_for_status()
    
    # xpath to find links in table data element
    html = lx.fromstring(response.text)
    links = html.xpath("//td/a")
    
    links_list = []
    param_check = "".join(["param=", param])
    units_check = "".join(["units=", units])
    year_check = "".join(["year=", str(year)])
    report_check = "report=SITE1YR"
    
    # for every link element from the xpath query, find the actual text of the link and
    # add it to the list if the link satistifies requirements for the link we want
    for link_element in links:
        link = link_element.attrib["href"]
        if param_check in link and units_check in link and year_check in link and report_check in link:
            links_list.append(link)
    return links_list


def get_all_links(param, units, start_year, end_year):
    """
    Given the parameter and the years, for each year, go to several AQMIS pages to retrieve the links to all sites,
    return all the links in a list
    """
    years = list(range(start_year, end_year + 1))
    all_links = []
    for year in years:
        url = "".join(["https://www.arb.ca.gov/aqmis2/display.php?param=", param, "&units=", units, "&year=", str(year), "&county_name=--COUNTY--&basin=--AIR+BASIN--&latitude=A-Whole+State&report=YRINV&order=s.name&submit=Retrieve+Data&ptype=aqd&std15="])
        list_of_links = read_links(param, units, year, url)
        for link in list_of_links:
            all_links.append(link)
    return all_links


def get_all_tables(links):
    """
    Given a list of AQMIS links, query the table from the link as a pandas data frame,
    each table being the yearly table from one site, and return a list of data frames
    """
    all_tables = []
    for link in links:
        url = "".join(["https://www.arb.ca.gov/aqmis2/", link])
        table = read_html_table(url)
        all_tables.append(table)
    return all_tables


def purge_bad_tables(list_of_tables):
    """
    Given a list of tables, each table being the yearly table from one site,
    remove the ones that do not have the right shape, return the list
    """
    for i in range(len(list_of_tables) - 1, -1, -1):
        if list_of_tables[i].shape != (36, 13):
            del list_of_tables[i]
    return list_of_tables

# OZONE tables

In [3]:
ozone_links = get_all_links("OZONE", "007", 2005, 2018)
ozone_tables = get_all_tables(ozone_links)
ozone_tables = purge_bad_tables(ozone_tables)
save_load.save_object(ozone_tables, "internal/ozone_tables.pkl")
ozone_tables = save_load.load_object("internal/ozone_tables.pkl")

save_load.save_object(ozone_links, "internal/ozone_links.pkl")

# PM2.5 tables

In [4]:
pm25_links = get_all_links("PM25", "001", 2005, 2018)
pm25_tables = get_all_tables(pm25_links)
pm25_tables = purge_bad_tables(pm25_tables)
save_load.save_object(pm25_tables, "internal/pm25_tables.pkl")
pm25_tables = save_load.load_object("internal/pm25_tables.pkl")

save_load.save_object(pm25_links, "internal/pm25_links.pkl")

# Data Wrangling (BY MONTH)

In [5]:
def flatten(table):
    """Given a table, return it in a row format"""
    row = []
    
    # get location name
    location_name = table[0][0].split("\xa0")[0].split(" Daily")[0]
    row.append(location_name)
    
    # get year
    year = table[0][0].split(" Data")[1].split(" Parts")[0].split(" Micrograms")[0]
    row.append(int(year))
    
    # get average per month, blank if month is empty
    for i in range(0, 12):
        daily = reformat_table(table).iloc[:,i]
        avgs = []
        for item in daily:
            if (str(item) != "nan"):
                avgs.append(float(item))
        if len(avgs) == 0:
            row.append("")
        else:
            row.append(np.mean(avgs))
    return row


def flatten_all(list_of_tables):
    """Given a list of tables, return a list of them flattened with flatten function"""
    list_of_rows = []
    for table in list_of_tables:
        list_of_rows.append(flatten(table))
    return list_of_rows


def create_col_names(start_year, end_year):
    """Return a list of the column names: every month from 2005 to 2018"""
    col_names = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            col_names.append("".join([str(month), "/", str(year)]))
    return(col_names)


def assemble(list_of_tables):
    """Given a list of tables, assemble the entire table"""
    # flatten, create location and month.year row and column names
    list_of_rows = flatten_all(list_of_tables)
    locations = pd.unique(pd.DataFrame(list_of_rows).iloc[:,0])
    col_names = create_col_names(2005, 2018)
    
    # fill up empty data frame with elements in list of rows
    df = pd.DataFrame(index = locations, columns = col_names)
    for row in list_of_rows:
        row_index = row[0]
        for i in range(1, 13):
            col_index = "".join([str(i), "/", str(row[1])])
            df.loc[row_index].loc[col_index] = row[1 + i]
    return df

# OZONE full

In [6]:
ozone = assemble(ozone_tables)
ozone.to_excel("datasets/ozone_month.xlsx")

# PM2.5 full

In [7]:
pm25 = assemble(pm25_tables)
pm25.to_excel("datasets/pm25_month.xlsx")